# Devoir 4 &mdash; Reconstruction 3D

## 0. Règles de codage

**<span style='color:Red'>
NE MODIFIEZ PAS LE CODE SQUELETTE EN DEHORS DES BLOCS TODO.<br>L'EVALUATEUR AUTOMATIQUE SERA TRES MECHANT AVEC VOUS SI VOUS LE FAITES !
</span>**

### 0.1. Résumé des fonctions potentiellement utiles (vous n'êtes pas obligé de les utiliser)  
- np.divide, 
- np.eye, 
- np.dot, 
- np.linalg.svd
- np.linalg.inv


### 0.2. Résumé des fonctions <span style='color:Red'>interdites</span>

- cv2.findFundamentalMat, cv2.computeCorrespondEpilines, cv2.decomposeEssentialMat, cv2.convertPointsToHomogeneous, 
- cv2.convertPointsFromHomogeneous, cv2.convertPointsHomogeneous, cv2.correctMatches, cv2.decomposeProjectionMatrix,
- cv2.solvePnP, cv2.solvePnPRansac, cv2.findHomography, cv2.initCameraMatrix2D, cv2.projectPoints, cv2.RQDecomp3x3,
- cv2.StereoBM_create, cv2.StereoSGBM_create, cv2.stereoCalibrate, cv2.StereoRectify, cv2.stereoRectifyUncalibrated,
- cv2.triangulatePoints, cv2.calibrateCamera, cv2.getOptimalNewCameraMatrix, scipy.linalg.rq,
- cv2.sfm.\*, skimage.feature.\*, skimage.measure.\*, skimage.transform.\*


Vous pouvez utiliser ces fonctions pour le débogage de votre code, mais la version finale ne doit en aucun cas les inclure faute d'avoir un zéro pour le devoir.

## 1. Aperçu

L'un des principaux domaines de la vision par ordinateur est la reconstruction 3D. Étant donné plusieurs images 2D d'un environnement, peut-on récupérer la structure 3D de l'environnement, ainsi que la position de la caméra / du robot ? Cela a de nombreuses utilisations en robotique et en systèmes autonomes car la compréhension de la structure 3D de l'environnement est cruciale pour la navigation. Vous ne voulez pas que votre robot heurte constamment les murs ou écrase votre chat !

<center>
<img src="figures/va_slam_full.png" style="width:70%"/> 
<br>
<b>Figure 1</b>. Exemple d'un robot utilisant SLAM, un algorithme de reconstruction et de localisation 3D
</center>


Dans ce devoir, il y a deux parties de programmation : la reconstruction de données clairsemées et la reconstruction dense. Les reconstructions clairsemées contiennent généralement peu de points, mais parviennent quand même à décrire les objets en question. Les reconstructions denses sont détaillées et à grain fin. Dans des domaines comme la modélisation 3D, des reconstructions denses extrêmement précises sont inestimables lors de la génération de modèles 3D d'objets et de scènes du monde réel.


Dans la **partie 1**, vous écrirez un ensemble de fonctions pour générer un nuage clairsemé de points pour certaines images de test fournies. Les images de test sont deux rendus (c.-à-d. images de synthèse) d'un temple grec sous deux angles différents. Un fichier python de type `npz` est également fourni et contient un ensemble de bonnes correspondances ponctuelles entre les deux images. Vous allez d'abord écrire une fonction qui calcule la matrice fondamentale entre les deux images. Ensuite, écrivez une fonction qui utilise la contrainte épipolaire pour trouver plus de paires de correspondance entre les deux images. Enfin, vous écrirez une fonction qui calculera les coordonnées des points 3D pour chaque paire de correspondances de points 2D.

<table><tr>
<td style="height:400px;"><img src="figures/im1.png" style="width:70%; transform: rotate(90deg);"/></td>
<td style="height:400px;"><img src="figures/im2.png" style="width:70%; transform: rotate(90deg);"/></td>
</tr></table>
<center><b>Figure 2</b>. Les deux images du temple fournies</center>

Nous vous avons également fourni d'autres fichiers `npz` utiles. Le fichier `data/some_corresp.npz` contient un ensemble de bonnes correspondances ponctuelles. Vous utiliserez ce fichier pour calculer la matrice fondamentale. Le fichier `data/intrinsics.npz` contient des matrices associées aux paramètres intrinsèques des caméras. Vous en aurez besoin pour calculer les matrices de projection des caméras. Enfin, le fichier `data/temple_coords.npz` contient les coordonnées de quelques points sur la première image qui devraient être faciles à localiser dans la seconde image.


Dans la **partie 2**, nous utilisons les paramètres extrinsèques calculés dans la **partie 1** pour réaliser une reconstruction 3D dense du temple. Pour cela, vous devez calculer les paramètres de rectification des images. Ensuite, utilisez la paire d’images rectifiées pour calculer une carte de disparité et une carte de profondeur dense.

In [ ]:
import cv2 as cv
import numpy as np
import helper as hlp
import submission as sub
import numpy.linalg as la
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

%matplotlib inline
#%matplotlib notebook
#%matplotlib ipympl
#%matplotlib nbagg
plt.rcParams['figure.figsize'] = (15.0, 12.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

## 2. Reconstruction clairsemée

Dans cette section, vous allez écrire un ensemble de fonctions pour calculer une reconstruction clairsemée d'un temple grec à partir de deux images. Vous commencerez par estimer la matrice fondamentale, ensuite vous calculerez les correspondances de points, et enfin afficher les résultats en 3D.


### 2.1.Algorithme à huit points (10 points)

Implémentez dans le fichier `submission.py` la fonction avec la signature suivante :

<p><center><code>F = eight_point( pts1, pts2, M)</code></center></p>

Cette fonction doit implémenter l'algorithme à huit points couvert en cours pour estimer la matrice fondamentale à partir d'un ensemble de points appariés. `pts1` et `pts2` sont des matrices $N \times 2$ correspondant respectivement aux coordonnées $\left(x, y \right)$ des $N$ points dans la première et la deuxième image. $M$ est un paramètre d'échelle. Voici les étapes à réaliser :

- Normaliser et dénormaliser la matrice $\mathbf{F}$ : D'un point de vue numérique, votre estimation de la matrice fondamentale peut être améliorée en normalisant les coordonnées des points appariés avant le calcul de la matrice fondamentale. Vous pouvez effectuer cette normalisation par des transformations linéaires pour rendre la magnitude moyenne des points approximativement égale à 1.0 ou un autre petit nombre ( $\sqrt{2}$ est également [recommandée](https://en.wikipedia.org/wiki/Eight-point_algorithm) ).

  $$
  \begin{bmatrix}
  u' \\
  v' \\
  1
  \end{bmatrix}
  =
  \underbrace{
  \begin{bmatrix}
  s  &  0  &  0 \\
  0  &  s  &  0 \\
  0  &  0  &  1
  \end{bmatrix}
  \begin{bmatrix}
  1  &  0  &  -c_u \\
  0  &  1  &  -c_v \\
  0  &  0  &  1
  \end{bmatrix}
  }_{\mathbf{T}}
  \begin{bmatrix}
  u \\
  v \\
  1
  \end{bmatrix}
  $$
  
  La matrice de transformation $\mathbf{T}$ est le produit des matrices de changement d'échelle et de translation. $c_u$ et $c_v$ sont les coordonnées moyennes. Il existe de nombreuses façons de choisir le paramètre $s$ pour l'échelle. Une façon simple consiste à définir $s$ égal à l'inverse du maximum entre la largeur et la hauteur de l'image. Quelle que soit la manière dont vous normalisez vos coordonnées, vous devez utiliser les matrices de transformation $\mathbf{T}_l$ (image de gauche) et $\mathbf{T}_r$ (image de droite) pour ajuster votre matrice fondamentale afin qu'elle puisse fonctionner sur les coordonnées de pixels d'origine :

  $$
  \mathbf{F}_{dénorm} = \mathbf{T}^T_l~\mathbf{F}_{norm}~\mathbf{T}_r
  $$


- Vous devez appliquer la contrainte de rang 2 sur $\mathbf{F}$ <ins>avant</ins> de dénormaliser. Rappelons qu'une matrice fondamentale $\mathbf{F}$ valide est celle où toutes les lignes épipolaires se croisent en un certain point. Mathématiquement, cela signifie qu'il existe un espace nul non trivial associé à $\mathbf{F}$. En général, et à cause d’imprécisions numériques dans le calcul des correspondances de points, l’algorithme à huit points produira une matrice $\mathbf{F}$ qui ne vérifie pas cette condition. Pour remédier à cela, décomposez $\mathbf{F}$ en valeurs singulières pour obtenir les trois matrices $\mathbf{U}$, $\mathbf{\Sigma}$, $\mathbf{V}$ tel que $\mathbf{F} = \mathbf{U}~\mathbf{\Sigma}~\mathbf{V}^T$. Convertissez ensuite la matrice $\mathbf{\Sigma}$ en une autre matrice $\mathbf{\Sigma}'$ de rang 2 en annulant la plus petite valeur singulière dans $\mathbf{\Sigma}$. Enfin, déduisez la nouvelle matrice fondamentale avec : $\mathbf{F}' = \mathbf{U}~\mathbf{\Sigma}'~ \mathbf{V}^T$.


<!-- - Il peut être utile d'affiner la solution en utilisant un algorithme de minimisation locale. Cela n'aidera probablement pas une solution qui ne fonctionne pas, mais peut améliorer une bonne solution en minimisant localement une fonction géométrique de coût. En ce sens, une fonction auxiliaire `refineF` est fournie dans `helper.py` qui prend $\mathbf{F}$ en entrée et les deux ensembles de points et renvoi une nouvelle matrice $\mathbf{F}$ améliorée. Vous devez appeler cette fonction avant de dénormaliser la matrice $\mathbf{F}$.
 -->

- Notez que « algorithme à huit points » n'est qu'un nom figuratif, cela signifie simplement que vous avez besoin d'au moins 8 points; votre algorithme doit utiliser un système surdéterminé (C.-à-d. plus de 8 points).


- Pour visualiser l'exactitude de votre matrice estimée $\mathbf{F}$, utilisez la fonction `displayEpipolarF` dans `helper.py` (doit être exécuté en dehors de ce notebook). Elle prend en entrée la matrice $\mathbf{F}$, et les deux images. Cette interface graphique vous permet de sélectionner un point dans l'image de gauche et de visualiser la ligne épipolaire correspondante dans l'image de droite (figure 3).

<center>
<img src="figures/epipolar.png" style="width:60%"/> <br>
<b>Figure 3</b>. Visualisation des lignes épipolaires avec <code>displayEpipolarF</code>
</center>

In [ ]:
# 1. Chargez en mémoire les images et les quelques correspondances de points 
#   fournies dans 'data/some_corresp.npz'

I1 = cv.cvtColor(cv.imread('./data/im1.png'), cv.COLOR_BGR2GRAY).astype(np.float32)
I2 = cv.cvtColor(cv.imread('./data/im2.png'), cv.COLOR_BGR2GRAY).astype(np.float32)

corresp = np.load('./data/some_corresp.npz')
pts1, pts2 = corresp['pts1'], corresp['pts2']

# 2. Exécutez la fonction 'eight_point' pour calculer la matrice fondamentale F.
r, c = I1.shape
M    = max(r,c)
F    = sub.eight_point(pts1, pts2, M)
print('Fundamental matrix F :\n', F)

### 2.2. Correspondances épipolaires (20 points)

Pour reconstruire une scène 3D avec une paire d'images stéréo, nous devons trouver de nombreuses correspondances de points. Une paire de correspondance associe les coordonnées des projections sur les deux images du même point 3D de la scène.

Avec un nombre suffisant de ces paires, une visualisation des points 3D estimés nous donnera un aperçu approximatif de l'objet. Dans vos devoirs précédents, vous avez trouvé des paires de points à l'aide de détecteurs et de descripteurs de primitives en comparant un point dans une image avec chaque point dans l'autre image. Mais ici, nous pouvons utiliser la matrice fondamentale pour simplifier considérablement cette recherche. 

<center>
<img src="figures/epipolargeo.jpg" style="width:40%"/> 
<br>
<b>Figure 4</b>. Géométrie épipolaire</center>


Etant donné un point $p_l$ dans une image (la vue de gauche sur figure 4). Son point de scène 3D correspondant $P$ pourrait se situer n'importe où le long de la ligne définie par le centre de la caméra $O_l$ et le point $p_l$. Cette ligne, avec le centre $O_r$ de la caméra d'une deuxième image (la vue de droite sur la figure 4) forme un plan. Ce plan coupe le plan image de la deuxième caméra, résultant en une ligne $l_r$ dans la deuxième image qui décrit tous les emplacements possibles que $p_r$ peut avoir ( c'est la projection de $P$ dans la deuxième image). La ligne $l_r$ est la ligne épipolaire, et il suffit de chercher le long de cette ligne pour trouver une correspondance possible pour le point $p_l$ dans la première image. 

Dans le fichier `submission.py`, implémentez la fonction avec la signature suivante :


<p><center><code>pts2 = epipolar_correspondences(I1, I2, F, pts1)</code></center></p>


où `I1` et `I2` sont les deux images de la paire stéréo, `F` est la matrice fondamentale calculée pour les deux images en utilisant votre fonction `eight_point`, `pts1` est une matrice $N \times 2$ contenant des points $\left(x, y\right)$ dans la première image. La fonction doit retourner `pts2`, une matrice $N \times 2$ qui contient les points correspondants dans la deuxième image. Voici les étapes à réaliser :


-	Pour chercher une correspondance possible pour un point $p_l$ dans l'image $I_1$, utilisez la matrice fondamentale pour estimer la ligne épipolaire correspondante $l_r$ et générer un ensemble de points candidats dans la deuxième image.


-	Pour chaque point candidat $p_c$, calculez un score de similitude entre $p_l$ et $p_c$. Le point parmi les candidats avec le score le plus élevé est considéré comme une correspondance épipolaire $p_r$.


-	Il existe de nombreuses façons de définir une fonction de similitude entre deux points. N'hésitez pas à utiliser ce que vous voulez comme fonction et à la **décrire dans votre rapport**. Une solution possible consiste à sélectionner une petite fenêtre de taille *w* autour du point $p_l$. Comparez ensuite cette fenêtre cible à la fenêtre du point candidat dans la deuxième image. Pour les images fournies, une simple distance euclidienne ou une [taxi-distance](https://fr.wikipedia.org/wiki/Distance_de_Manhattan) devrait suffire.


Vous pouvez utiliser la fonction `epipolarMatchGUI` dans `helper.py` pour tester visuellement votre fonction (doit être éxécutée en dehors de ce notebook). Votre fonction n'a pas besoin d'être parfaite, mais elle devrait correctement obtenir la plupart des points faciles, comme les coins, les points singuliers, etc.

**Dans votre rapport** : Veuillez inclure une capture d'écran de `epipolarMatchGUI` en cours d'exécution avec votre implémentation de `epipolar_correspondences` (similaire à la figure 5). Mentionnez la métrique de similitude que vous avez utilisée. Commentez également tous les cas où votre algorithme de correspondance échoue en suggérant une raison éventuelle de l’échec. 

<center>
<img src="figures/match.png" style="width:60%"/> 
<br>
<b>Figure 5</b>: Visualisation de correspondances épipolaires avec <code>epipolarMatchGUI</code>.
</center>

In [ ]:
# Chargez en mémoire les points de l'image 1 contenus dans 'data/temple_coords.npz' 
# et exécuter la fonction 'epipolar_correspondences' sur ces points pour obtenir 
# leurs homologues dans l’image 2

coords = np.load('./data/temple_coords.npz')
pts1p = coords['pts1']
pts2p = sub.epipolar_correspondences(I1, I2, F, pts1p)

### 2.3. La matrice Essentielle (5 points)

Afin d'obtenir les matrices de projection $\mathbf{P}$ de la caméra, nous devons calculer la matrice essentielle. Implémentez dans le fichier `submission.py` la fonction avec la signature suivante :

<p><center><code>E = essential_matrix(F, K1, K2)</code></center></p>

où `F` est la matrice fondamentale déjà calculée, `K1` et `K2` sont les matrices intrinsèques des caméras pour la première et la deuxième image, respectivement (contenues dans le fichier `data/intrinsics.npz`), et `E` est la matrice essentielle retrouvée. Les paramètres intrinsèques d’une caméra sont généralement acquis par étalonnage de celle-ci. Reportez-vous aux slides du cours pour la relation entre la matrice fondamentale et la matrice essentielle.

**Dans votre rapport** : Veuillez inclure votre matrice estimée $\mathbf{E}$ pour la paire d'images du temple.

In [ ]:
# Charger en mémoire le fichier 'data/intrinsics.npz' et calculer la matrice
# essentielle E.

intrinsics = np.load('./data/intrinsics.npz')
K1, K2 = intrinsics['K1'], intrinsics['K2']
E = sub.essential_matrix(F, K1, K2)
print('Essential matrix E :\n', E)

### 2.4. Triangulation (10 points)

Implémentez dans le fichier `submission.py` la fonction avec la signature suivante :

<p><center><code>pts3d = triangulate(P1, pts1, P2, pts2)</code></center></p>

Cette fonction devrait trianguler des paires de points 2D dans les images en un ensemble de points 3D. `pts1` et `pts2` sont les matrices $N \times 2$ contenant les coordonnées de points 2D dans les images, `P1` et `P2` sont les matrices de projection de caméra $3 \times 4$ et `pts3d` est une matrice $N \times 3$ contenant les coordonnées des points 3D correspondants. 

Les slides du cours présentent le processus de triangulation pour le cas de caméras parallèles... ici, vous devez implémenter un algorithme de triangulation pour le cas général. Ainsi, étant donné deux matrices de projection $\mathbf{P}_1$ et $\mathbf{P}_2$ et une paire de correspondance $\mathbf{p}_l \leftrightarrow \mathbf{p}_r$ pour un point 3D $\mathbf{X}$ :

$$
w_l \mathbf{\tilde{p}}_l = \mathbf{P}_1\,\mathbf{\tilde{X}},\quad
w_r \mathbf{\tilde{p}}_r = \mathbf{P}_2\,\mathbf{\tilde{X}},\quad
\mathbf{\tilde{p}}_l=
\begin{bmatrix}
u^l\\
v^l\\
1
\end{bmatrix},\quad
\mathbf{\tilde{p}}_r=
\begin{bmatrix}
u^r\\
v^r\\
1
\end{bmatrix},\quad
\mathbf{P}_i=
\begin{bmatrix}
\rule[.5ex]{2.5ex}{0.5pt} \left( \mathbf{p}^i_1 \right)^T \rule[.5ex]{2.5ex}{0.5pt} \\
\rule[.5ex]{2.5ex}{0.5pt} \left( \mathbf{p}^i_2 \right)^T \rule[.5ex]{2.5ex}{0.5pt} \\
\rule[.5ex]{2.5ex}{0.5pt} \left( \mathbf{p}^i_3 \right)^T \rule[.5ex]{2.5ex}{0.5pt} 
\end{bmatrix}
$$

où $w_l$ et $w_r$ sont des constantes quelconques non nulles (rappelons que les matrices $\mathbf{P}_1$ et $\mathbf{P}_2$ sont définies à des facteurs multiplicatifs près). De ce fait, nous avons 

$$
u^l \left( \mathbf{p}^1_3 \right)^T \mathbf{\tilde{X}} = \left( \mathbf{p}^1_1 \right)^T \mathbf{\tilde{X}}\quad, \quad
u^r \left( \mathbf{p}^2_3 \right)^T \mathbf{\tilde{X}} = \left( \mathbf{p}^2_1 \right)^T \mathbf{\tilde{X}} \\
v^l \left( \mathbf{p}^1_3 \right)^T \mathbf{\tilde{X}} = \left( \mathbf{p}^1_2 \right)^T \mathbf{\tilde{X}}\quad , \quad
v^r \left( \mathbf{p}^2_3 \right)^T \mathbf{\tilde{X}} = \left( \mathbf{p}^2_2 \right)^T \mathbf{\tilde{X}}
$$

sous forme matricielle, cela donne :

$$
\mathbf{D}\,\mathbf{\tilde{X}} = 0, \quad
\mathbf{D}=
\begin{bmatrix}
u^l \left( \mathbf{p}^1_3 \right)^T - \left( \mathbf{p}^1_1 \right)^T \\
v^l \left( \mathbf{p}^1_3 \right)^T - \left( \mathbf{p}^1_2 \right)^T \\
u^r \left( \mathbf{p}^2_3 \right)^T - \left( \mathbf{p}^2_1 \right)^T \\
v^r \left( \mathbf{p}^2_3 \right)^T - \left( \mathbf{p}^2_2 \right)^T
\end{bmatrix},\quad
\mathbf{D} \in \mathbb{R}^{4\times4} ,\quad \mathbf{\tilde{X}} \in \mathbb{R}^4 
$$

Le vecteur homogène $\mathbf{\tilde{X}}$ représente donc un vecteur singulier droite de la matrice $\mathbf{D}$ et peut être obtenu par décomposition SVD de celle-ci. N'oubliez pas de diviser le vecteur $\mathbf{\tilde{X}}$ par sa quatrième composante pour récupérer les coordonnées cartésiennes du point 3D $\mathbf{X}$.  


### 2.5. Matrices extrinsèques  (15 points)


Implémentez dans le fichier `submission.py` la fonction avec la signature suivante :

<p><center><code>R1, t1, R2, t2, pts3d, err = ComputeExtrinsic( K1, K2, E, pts1, pts2 )</code></center></p>


Cette fonction permet de retouver les paramètres extrinsèques des deux caméras `(R1,t1)` et `(R2,t2)` ainsi que les coordonnées de points 3D `pts3d` à partir d'une liste de paire de correspondances 2D `pts1` et `pts2`.

`K1` et `K2` sont les matrices intrinsèques des caméras pour la première et la deuxième image, respectivement. `E` est la matrice essentielle. `pts1` et `pts2` sont des matrices $N \times 2$ contenant les coordonnées de points 2D dans les deux images. `R1` et `R2` sont des matrices de rotation $3 \times 3$ des caméras de gauche et de droite, respectivement. `t1` et `t2` sont des vecteurs de translation $3 \times 1$ associés aux caméras. `pts3d` est une matrice $N \times 3$ contenant les coordonnées des points 3D calculés, et `err` est l'erreur de reprojection. 

Pour calculer les points `pts3d`, utilisez votre fonction `triangulate` implémentée dans la section précédente. En ce sens, vous aurez besoin des matrices de projection $\mathbf{P}_1$ et $\mathbf{P}_2$. Rappelons qu'une matrice de projection est le produit des matrices intrinsèque et extrinsèque de la caméra.

Pour $\mathbf{P}_1$, aucune rotation ou translation est nécessaire, donc la matrice extrinsèque est simplement $\left[\, \mathbf{I}\, |\, \mathbf{0}\, \right]$. Pour $\mathbf{P}_2$, passez la matrice essentielle $\mathbf{E}$ à la fonction `camera2` fournie dans `helper.py` pour obtenir quatre matrices extrinsèques $\left[\,\mathbf{R}_i\, | \,\mathbf{t}_i\,\right]$ possibles. Vous devez déterminer laquelle d'entre ces matrices est le [bon choix](https://en.wikipedia.org/wiki/Essential_matrix#Determining_R_and_t_from_E) pour $\mathbf{P_2}$. En ce sens, utilisez les quatre combinaisons possibles de $\mathbf{P_1}$ et $\mathbf{P_2}$ pour exécuter votre fonction `triangulate` sur l’ensemble des points `pts1` et leurs correspondances `pts2`. La bonne matrice extrinsèque $\left[\,\mathbf{R}_2\, | \,\mathbf{t}_2\,\right]$ est celle où la plupart des points 3D `pts3d` se trouvent devant les deux caméras (profondeur positive).

Enfin, vérifiez les performances de votre implémentation en examinant l'erreur de re-projection. Pour calculer l'erreur de re-projection, les points 3D estimés `pts3d` doivent être projetés sur l'image `I1`. Ensuite, la distance euclidienne moyenne est calculée entre les projections obtenues et les données fournies dans `pts1`.

**Dans votre rapport** : 
  1. Décrivez comment vous avez déterminé quelle matrice extrinsèque est correcte. Notez qu'il ne suffit pas de reformuler l’indice donné ici. Signalez votre erreur de re-projection. Si votre implémentation est correcte, l'erreur de re-projection doit être inférieure à 1 pixel.

  2. Utilisez la fonction `scatter` de `matplotlib` pour visualiser un nuage de points sur l'écran. Incluez trois images de votre reconstruction sous différents angles comme illustré dans figure 6.

<center>
<img src="figures/recplotscatter.png" style="width:50%"/> 
<br>
<b>Figure 6</b>. Exemple de reconstructions
</center>

In [ ]:
#R1, t1, R2, t2, pts3d, err = sub.ComputeExtrinsic( K1, K2, E, pts1 , pts2  )
R1, t1, R2, t2, pts3d, err = sub.ComputeExtrinsic( K1, K2, E, pts1p, pts2p )

# Si votre implémentation est correcte, l'erreur de re-projection doit être inférieure à 1 pixel.
print('Reprojection Error: ', err)


# Utilisez la fonction 'scatter' de 'matplotlib' pour visualiser les reconstructions  
# sur l'écran - vous aurez besoin de ces images pour votre rapport.
#

# implémentez cela ici


## 3. Reconstruction dense

Dans des applications telles que la modélisation 3D, l'impression 3D, Réalité Augmentée et Réalité Virtuelle, un modèle de points clairsemés ne suffit pas. Lorsque les utilisateurs visualisent la reconstruction, il est beaucoup plus agréable de gérer une reconstruction dense. En ce sens, il est essentiel de rectifier les images pour faciliter l’étape de calcul des correspondances.


Dans cette section, vous allez écrire un ensemble de fonctions pour effectuer une reconstruction dense sur l’exemple d’images du temple. En utilisant les paramètres intrinsèques et extrinsèques des caméras de l’étape précédente, vous devez écrire une fonction pour calculer les paramètres de rectification pour les deux images. Rappelons que les lignes épipolaires dans les images rectifiées sont horizontales, ce qui simplifie considérablement la recherche de correspondances pour la reconstruction 3D. Enfin , vous calculerez les cartes de disparité et de profondeur.


### 3.1. Rectification des images (10 points)


Implémentez dans le fichier `submission.py` la fonction avec la signature suivante :

<p><center><code>H1, H2, K1p, K2p, R1p, R2p, t1p, t2p = rectify_pair(K1, K2, R1, R2, t1, t2)</code></center></p>


Cette fonction calcule les matrices de rectification `H1` et `H2` pour les deux images. Elle prend en entrée les paramètres `(K1, R1, t1)` et `(K2, R2, t2)` des caméras gauche et droite et renvoie les matrices de rectification `H1` et `H2` ainsi que les paramètres des caméras modifiés `(K1p, R1p, t1p, K2p, R2p, t2p)`. 

Suivant la méthode illustée dans le livre de Trucco & Verri, "Introductory Techniques for 3-D Computer Vision", pp. 157-161, la fonction de rectification doit exécuter consécutivement les étapes suivantes :

1.	Calculez les centres optiques $O_l$ et $O_r$ des caméras avec les équations : $O_i = - \mathbf{R}_i^{-1}\mathbf{t}_i, \, i \in \{1,2\}.$<br><br>

2. Calculez la matrice de rectification $\mathbf{R_{rect}} = \left[\mathbf{r_1},\, \mathbf{r_2},\, \mathbf{r_3} \right]^T $ où $\left\{\mathbf{r_1},\, \mathbf{r_2},\, \mathbf{r_3}\right\} \in \mathbb{R}^{3 \times 1}$ sont des vecteurs orthonormaux qui représentent respectivement les axes $x$, $y$ et $z$ du repère de la caméra rectifiée.<br>
  
   (a) Le nouvel axe $x$ ($\mathbf{r_1}$) est parallèle à la ligne de base et est donné par l'épipole; puisque le centre optique est à l'origine, $\mathbf{r}_1$ coïncide avec la direction de translation. C.-à-d. $\mathbf{r}_1 = \mathbf{t}_2 / \| \mathbf{t}_2 \|$
  
   (b) Le nouvel axe $y$ ($\mathbf{r_2}$) est orthogonal à l'axe $x$ et choisi de façon qu'il soit aussi orthogonal à l'ancien axe $z$ de la caméra de gauche. C.-à-d. $\mathbf{r_2}$ est le produit vectoriel de $\mathbf{R_1}\left[3,:\right]^T$ et $\mathbf{r_1}$.
  
   (c) Le nouvel axe $z$ ($\mathbf{r_3}$) est orthogonale aux axes $x$ et $y$. C.-à-d. $\mathbf{r_3}$ est le produit vectoriel de $\mathbf{r_1}$ et $\mathbf{r_2}$.
  <br><br>
3. Définissez les nouvelles matrices de rotation : $\mathbf{R}'_1 = \mathbf{R}'_2 = \mathbf{R_{rect}}$. <br><br>

4. Calculez les nouveaux paramètres intrinsèques : $\mathbf{K}'_1 = \mathbf{K}'_2 = {\mathbf{K}}_2$. <br><br>

5. Calculez les nouveaux vecteurs de translation : $\mathbf{t}'_1 = -\mathbf{R}'_1\,O_l$ et $\mathbf{t}'_2 = -\mathbf{R}'_2\,O_r$.<br><br>

6. Enfin, les matrices de rectification des images $\mathbf{H}_1$ et $\mathbf{H}_2$ sont obtenues par les équations : $\mathbf{H}_i = \mathbf{K}'_i \, \mathbf{R}_i \, \mathbf{R_{rect}}\, \mathbf{K}_i^{-1}, \, i \in \{1,2\}.$

Une fois votre implémantation terminée, testez votre fonction en utilisant le script ci-dessous. Ce script test votre code de rectification sur les images du temple en utilisant les paramètres intrinsèques fournis et vos paramètres extrinsèques calculés. 

Veuillez notez que ce n'est pas l'unique façon de rectifier une paire d'images stéréos, l'article de [Loop and Zhang](https://drive.google.com/file/d/1x2DaxSAwOUVG8VORU5ZE1JRzW6aGYOLM/view?usp=sharing) présente une autre approche interessante.

**Dans votre rapport** : incluez une capture d'écran des plots de ce script. Les résultats devraient montrer quelques lignes épipolaires qui sont parfaitement horizontales, avec des points correspondants dans les deux images se trouvant sur la même ligne.

In [ ]:
# 1. Rectify the images and save the parameters
H1, H2, K1p, K2p, R1p, R2p, t1p, t2p = sub.rectify_pair(K1, K2, R1, R2, t1, t2)


# 2. Warp the images
I1p, I2p, T1, T2 = hlp.warpStereo(I1, I2, H1, H2)

# 3. display the result
r, c = I1p.shape
I = np.zeros((r, 2*c))
I[:,:c] = I1p
I[:,c:] = I2p

pts1pp, pts2pp = corresp['pts1'][::18].T, corresp['pts2'][::18].T
pts1pp, pts2pp = hlp._projtrans(T1 @ H1, pts1pp), hlp._projtrans(T2 @ H2, pts2pp)

pts2pp[0,:] = pts2pp[0,:] + c

plt.imshow(I, cmap='gray')
plt.scatter(pts1pp[0,:], pts1pp[1,:], s=60, c='b', marker='*')
plt.scatter(pts2pp[0,:], pts2pp[1,:], s=60, c='b', marker='*')

for p1, p2 in zip(pts1pp.T, pts2pp.T):
    plt.plot([p1[0],p2[0]], [p1[1],p2[1]], '-', c='r')  # red  line   - your line 
    plt.plot([p1[0],p2[0]], [p1[1],p1[1]], '-', c='y')  # yellow line - truly horizontal
    
plt.show()


### 3.2. Carte de disparité (20 points)

Implémentez dans le fichier `submission.py` la fonction avec la signature suivante :

<p><center><code>dispM = get_disparity (im1, im2, max_disp, win_size)</code></center></p>

Cette fonction calcule la carte de disparité à partir d'une paire d'images rectifiées `im1` et `im2`. Le paramètre `max_disp` est la disparité maximale autorisée et `win_size` est la taille de la fenêtre. La sortie `dispM` a la même dimension que les images `im1` et `im2`. Comme `im1` et `im2` sont des images rectifiés, le calcul des correspondances est réduit à un problème de recherche 1D. La valeur de `dispM(y, x)` est donnée par :

$$
\displaystyle{
\mathrm{dispM}\left(y, x\right) = \argmin_{0 \leq d \leq \max disp } \mathrm{dist}\left( \mathrm{im1}\left(y,x\right), \mathrm{im2}\left(y,x - d\right) \right)
}
$$

où

$$
\displaystyle{
\mathrm{dist}\left( \mathrm{im1}\left(y,x\right), \mathrm{im2}\left(y,x - d\right) \right)= \sum_{i=-w}^{w} \sum_{j=-w}^{w} \left( \mathrm{im1}\left(y + i,x + j\right) - \mathrm{im2}\left(y + i , x + j - d\right)  \right)^2
}
$$

et $w = \left( \mathrm{win\_size}-1\right)/2$. Cette somme sur la fenêtre peut être facilement calculée en utilisant la fonction `scipy.signal.convolve2d` (c'est-à-dire une convolution avec un masque de uns). Notez que ce n'est pas la seule façon d'implémenter cette opération.


### 3.3. Carte de profondeur  (10 points)

Implémentez dans le fichier `submission.py` la fonction avec la signature suivante :

<p><center><code>depthM = get_depth (dispM, K1, K2, R1, R2, t1, t2)</code></center></p>

Cette fonction crée une carte de profondeur à partir d'une carte de disparité. Rappelons la relation vue en cours entre la disparité et la profondeur

$$
\displaystyle{
\mathrm{depthM}\left(y, x\right) = \frac{b \cdot f }{\mathrm{dispM}\left(y, x\right)} 
}
$$

où $b$ est la longueur de la ligne de base et $f$ est la distance focale de la caméra. Pour simplier, supposez que $f=\mathbf{K}_1\left(1,1\right)$.

Executez le script ci-dessous pour tester vos fonctions pour le calcul des cartes de disparité et de profondeur.


**Dans votre rapport** : Veuillez inclure des images des cartes de disparité et de profondeur.


In [ ]:
# 1. Get disparity and depth maps

max_disp, win_size = 20, 3

dispM = sub.get_disparity(I1, I2, max_disp, win_size)

global_disp = T2[0,-1] - T1[0,-1]
dispM = dispM + global_disp

depthM = sub.get_depth(dispM, K1p, K2p, R1p, R2p, t1p, t2p)

dismin, dismax = ( np.amin(  dispM  ) , np.amax(  dispM  ) )
depmin, depmax = ( np.amin(  depthM ) , np.amax(  depthM ) )

# 2. Display disparity and depth maps

dispI  = ( ( dispM  - dismin) / ( dismax - dismin )  ) * (I1>40) 
depthI = ( ( depthM - depmin) / ( depmax - depmin )  ) * (I1>40)

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(dispI, cmap='gray')
ax1.set_title('Disparity Image')
ax1.set_axis_off()
ax2.imshow(depthI, cmap='gray')
ax2.set_title('Depth Image')
ax2.set_axis_off()
plt.tight_layout()
plt.show()


## 4. Estimation de pose (Tâches bonus)

Dans cette section, vous mettrez en œuvre ce que vous avez appris en cours pour estimer les paramètres intrinsèques et extrinsèques d’une caméra étant donné des points 2D dans l’image et leurs points correspondants dans la scène 3D. En d'autres termes, étant donné un ensemble de points appariés ${\mathbf{X}_i, \mathbf{x}_i}$ et un modèle de caméra 
$$
w
\begin{bmatrix}
\mathbf{x} \\
1
\end{bmatrix} 
=
\mathbf{P}
\begin{bmatrix}
\mathbf{X} \\
1
\end{bmatrix}
.
$$

Nous voulons estimer la matrice de projection $\mathbf{P} \in \mathbb{R}^{3 \times 4}$, ainsi que les paramètres intrinsèques $\mathbf{K} \in \mathbb{R}^{3 \times 3}$, matrice de rotation $\mathbf{R} \in \mathbb{R}^{3 \times 3}$ et la translation $\mathbf{t} \in \mathbb{R}^3$ de la caméra, telles que
$$
\mathbf{P} = \mathbf{K}\left[\,\mathbf{R}\,|\,\mathbf{t}\,\right]
$$

### 4.1. Estimation de la matrice de projection P	(5 points)

Implémentez dans le fichier `submission.py` la fonction avec la signature suivante :

<p><center><code>P = estimate pose(x, X)</code></center></p>

Cette fonction estime la matrice de projection $\mathbf{P}$ étant donné des points 2D et 3D. $\mathbf{x}$ est une matrice $2 \times N$ indiquant les coordonnées $(x, y)$ des $N$ points 2D sur le plan de l'image et $\mathbf{X}$ est une matrice $3 \times N$ désignant les coordonnées $(x, y, z)$ des points correspondants dans un repère 3D. Rappelons que la matrice $\mathbf{P}$ peut être calculée en utilisant la même stratégie utilisée lors de l'estimation d'homographies déjà vue dans les devoirs précédents. 

Vous pouvez exécuter le script ci-dessous pour tester votre implémentation.

**Dans votre rapport** : Inclure <ins>la sortie</ins> du script.

In [ ]:
# 1. Generate random camera matrix

K = np.array([[1,0,100], [0,1,100], [0,0,1]])
R, _,_ = la.svd(np.random.randn(3,3))
if la.det(R) < 0: R = -R
t = np.vstack((np.random.randn(2,1), 1))

P = K @ np.hstack((R, t))


# 2. Generate random 2D and 3D points

N = 100

X = np.random.randn(N,3)
x = P @ np.hstack((X, np.ones((N,1)))).T
x = x[:2,:].T / np.vstack((x[2,:], x[2,:])).T


# 3. Test pose estimation with clean points

Pc = sub.estimate_pose(x, X)

xp = Pc @ np.hstack((X, np.ones((N,1)))).T
xp = xp[:2,:].T / np.vstack((xp[2,:], xp[2,:])).T

print('Reprojection Error with clean 2D points:', la.norm(x-xp))
print('Pose Error with clean 2D points:', la.norm((Pc/Pc[-1,-1])-(P/P[-1,-1])))


# 4. Test pose estimation with noisy points

x = x + np.random.rand(x.shape[0], x.shape[1])
Pn = sub.estimate_pose(x, X)

xp = Pn @ np.hstack((X, np.ones((N,1)))).T
xp = xp[:2,:].T / np.vstack((xp[2,:], xp[2,:])).T

print('Reprojection Error with noisy 2D points:', la.norm(x-xp))
print('Pose Error with noisy 2D points:', la.norm((Pn/Pn[-1,-1])-(P/P[-1,-1])))


### 4.2. Estimation des paramètres intrinsèques / extrinsèques	(5 points)

Implémentez dans le fichier `submission.py` la fonction avec la signature suivante :

<p><center><code>K, R, t = estimate_params(P)</code></center></p>


Cette fonction calcule les paramètres intrinsèques et extrinsèques à partir de la matrice de projection 

$$\mathbf{P}\triangleq \mathbf{K} \left[ \mathbf{R} \,|\, \mathbf{t} \right] = \left[ \underbrace{\mathbf{K} \mathbf{R}}_{\mathbf{M}} \,|\, \mathbf{K} \mathbf{t} \right] = \underbrace{\mathbf{K}\mathbf{R}}_{\mathbf{M}} \left[ \mathbf{I} \,|\, \underbrace{ \mathbf{R}^T\mathbf{t}}_{-\mathbf{c}} \right].$$ 


En ce sens, il est possible de procéder de la façon suivante (d'autres méthodes existent) :

1.	Calculez le centre $\mathbf{c}$ de la caméra en utilisant la décomposition en valeurs singulières SVD de $\mathbf{P}$. Indication : $\mathbf{c}$ est le vecteur associé à la plus petite valeur singulière.<br><br>

2.	Estimatez la matrice intrinsèque $\mathbf{K}$ et la matrice de rotation $\mathbf{R}$ en utilisant la décomposition **QR** de la matrice $\mathbf{M}$. Rappelons que la matrice $\mathbf{K}$ est triangulaire supérieure droite tandis que $\mathbf{R}$ est une matrice orthonormée.<br><br>

3.	Calculez le vecteur de translation avec : $\mathbf{t} = -\mathbf{R}\,\mathbf{c}$.<br><br>


Vous pouvez exécuter le script ci-dessous pour tester votre implémentation.

**Dans votre rapport** : Inclure <ins>la sortie</ins> du script.


In [ ]:
# 3. Test parameter estimation with clean points
Kc, Rc, tc = sub.estimate_params(Pc)

print('Intrinsic Error with clean 2D points:', la.norm((Kc/Kc[-1,-1])-(K/K[-1,-1])))
print('Rotation Error with clean 2D points:', la.norm(R-Rc))
print('Translation Error with clean 2D points:', la.norm(t-tc))

# 4. Test parameter estimation with noisy points
Kn, Rn, tn = sub.estimate_params(Pn)

print('Intrinsic Error with noisy 2D points:', la.norm((Kn/Kn[-1,-1])-(K/K[-1,-1])))
print('Rotation Error with noisy 2D points:', la.norm(R-Rn))
print('Translation Error with noisy 2D points:', la.norm(t-tn))


### 4.3. Projection d'un modèle CAO sur l'image (10 points) 

Vous allez maintenant utiliser ce que vous avez implémenté pour estimer la matrice de projection associée à une image réelle illustrée dans la figure 7 (gauche), et projeter sur cette image l'objet 3D (modèle CAO) illustré dans la figure 7 (droite). 

<table><tr>
<td><img src="figures/aeroplane.jpg" style="width:80%"/></td>
<td><img src="figures/aeroplane_cad.jpg" style="width:80%"/></td>
</tr></table>
<center><b>Figure 7</b>. L'image et le modèle CAO fournis.</center>


Implémentez ci-dessous un script pour faire ce qui suit :

1.	Chargez l'image « **image** », le modèle CAO « **cad** », les points 2D « **x** » et les points 3D « **X** » à partir du fichier de données `data/pnp.npz`. Cette étape est déjà implémentée pour vous.<br><br>

2.	Exécutez les fonctions `estimate_pose` et `estimate params` pour calculer la matrice de projection $\mathbf{P}$, et les matrices $\mathbf{K}$ et $\mathbf{R}$ ainsi que  le vecteur de translation $\mathbf{t}$.<br><br>

3.	Utilisez votre matrice $\mathbf{P}$ pour projeter sur l'image les points 3D fournis « **X** ».<br><br>

4.	Visualiser sur l’écran les points 2D fournis « **x** » et les projections des points 3D. Un exemple est illustré à la figure 8 (gauche).<br><br>

5.	Visualisez sur l’écran le modèle CAO aligné par votre matrice rotation $\mathbf{R}$. Un exemple est illustré à la figure 8 (milieu).<br><br>

6.	Projetez tous les sommets du modèle CAO sur le plan de l'image et dessinez ensuite le modèle CAO projeté par-dessus l'image 2D. Un exemple est illustré à la figure 8 (droite).<br><br>


**Dans votre rapport** : Inclure trois images similaires à la figure 8. **Vous devez utiliser des couleurs différentes** de la figure 8. Par exemple, un cercle vert pour les points 2D fournis, des points noirs pour les points 3D projetés, un tracé bleu du modèle CAO et un modèle CAO projeté rouge se chevauchant sur l'image. Vous n'obtiendrez **AUCUN** point si vous utilisez la même couleur.


<table><tr>
<td><img src="figures/aeroplane_example1.jpg" style="width:100%"/></td>
<td><img src="figures/aeroplane_example2.jpg" style="width:100%"/></td>
<td><img src="figures/aeroplane_example3.jpg" style="width:100%"/></td>
</tr></table>
<center><b>Figure 8</b>. Projection d'un modèle CAO sur l'image. <br> (gauche) : l'image annotée avec des points 2D fournis (cercle bleu) et des points 3D projetés (points rouges) <br>(milieu) : Le modèle CAO pivoté avec la matrice <b>R</b> <br>(droite) : le modèle CAO projeté sur l'image.</center>


In [ ]:
# 1. Chargez l'image « image », le modèle CAO « cad », les points 2D « x » 
#    et les points 3D « X » à partir du fichier de données data/pnp.npz
projectcad = np.load('./data/pnp.npz', allow_pickle=True)
image, cad, x, X = projectcad['image'], projectcad['cad'],  projectcad['x'],  projectcad['X']

cad = cad.tolist()

vertices  = cad['vertices']    #  coordonnées des sommets du model CAO
triangles = cad['faces']       #  facettes (indices de trois sommets par facette) du model CAO


# TODO 4.3 : Estimer la matrice de projection à partir d'une image réelle 
#            et projeter sur cette image un modèle CAO
# TODO-BLOC-DEBUT    
# 2. Exécutez les fonctions 'estimate_pose' et 'estimate params' pour calculer 
#    la matrice de projection P, les matrices K et R  ainsi que le vecteur de translation t.
# 3. Utilisez votre matrice P pour projeter les points 3D fournis « X » sur l'image.
# 4. Visualiser sur l’écran les points 2D fournis « x » et les projections des points 3D. 
# 5. Visualisez sur l’écran le modèle CAO aligné par votre matrice rotation R. 
# 6. Projetez tous les sommets du modèle CAO sur le plan de l'image et dessinez ensuite 
#     le modèle CAO projeté par-dessus l'image 2D.
raise NotImplementedError("TODO 4.3 : non implémenté")
# TODO-BLOC-FIN    

<del>

## 5. Livrables

### 5.1. Le code (à remettre en classe)

Un fichier zip contenant le fichier **`submission.py`** et ce notebook modifié (section 2.5 et éventuellement 4.3 si vous avez implémenté les taches bonus)

**Le code sera remis en classe pendant votre séance de TP au serveur INGInious - <span style='color:Red'> aucun document ou code ne sera accepté si envoyé par mail ou clé USB</span>**.

### 5.2. Le rapport (à remettre sur Google Classroom)

Éditez un court rapport qui décrit **clairement** votre travail. Les éléments à inclure dans ce document sont indiqués dans la rubriques "**Rapport**" dans chaque question de ce notebook. Veuillez noter que les scans de document manuscript ne seront **<span style='color:Red'>PAS</span>** acceptés.

</del>